In [1]:
import pandas as pd
import seaborn as sns
import pickle
import numpy as np
import catboost as ctb

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from catboost import CatBoostClassifier

In [3]:
%matplotlib inline

In [4]:
delta_time=10

In [5]:
circle_radius=50

In [6]:
file_name = "Final Summary - STAR - DXB - ({rd:.0f} NM) Revision 03 ({dt:.1f} minutes).csv"
file_name=file_name.format(dt=delta_time,rd=circle_radius)

In [7]:
df = pd.read_csv(file_name,low_memory=False)

In [8]:
df.columns

Index(['DATE', 'ICAO24', 'FILE', 'PROCEDURE', 'TOUCHDOWN TIME',
       'LANDING RUNWAY', 'ENTRY POINT @ STAR',
       'LONGITUDE - ENTRY 50 NM RADIUS', 'LATITUDE - ENTRY 50 NM RADIUS',
       'TIME UTC - ENTRY 50 NM RADIUS', 'FLAG - ESTIMATED ENTRY 50 NM RADIUS',
       'QUADRANT', 'DISTANCE OF THE CLOSEST POINT TO 50 NM RADIUS (NM)',
       'RECOMPUTED TOUCHDOWN TIME',
       'MINIMUM DISTANCE TO RUNWAY THRESHOLD (NM)',
       'DELTA TOUCHDOWN TIME @ SAME RUNWAY (PREVIOUS LANDING) (MIN.)',
       'WAKE TURBULENCE CATEGORY LANDING SEQUENCE @ SAME RUNWAY (PREVIOUS-CURRENT)',
       'FLIGHT NUMBER', 'STA', 'DELTA STA (MIN.)',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE',
       'NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (1ST QUADRA

In [9]:
file_name_f="Final Summary - STAR - DXB - ({rd:.0f} NM) Revision 03 ({dt:.1f} minutes).f"
file_name_f=file_name_f.format(dt=delta_time,rd=circle_radius)
column_1="TIME UTC - ENTRY {rd:.0f} NM RADIUS"
column_1=column_1.format(rd=circle_radius)
column_2="FLAG - ESTIMATED ENTRY {rd:.0f} NM RADIUS"
column_2=column_2.format(rd=circle_radius)
column_3="DISTANCE OF THE CLOSEST POINT TO {rd:.0f} NM RADIUS (NM)"
column_3=column_3.format(rd=circle_radius)
column_4="DELTA TIME ENTRY @ {rd:.0f} NM CIRCLE FROM PREVIOUS FLIGHT IN THE SAME QUADRANT (MIN.)"
column_4=column_4.format(rd=circle_radius)
df.drop(columns=["DATE","ICAO24","FILE","PROCEDURE","FLIGHT NUMBER","STA","DELTA STA (MIN.)","TOUCHDOWN TIME",column_1,column_2,column_3,"RECOMPUTED TOUCHDOWN TIME","MINIMUM DISTANCE TO RUNWAY THRESHOLD (NM)","DELTA TOUCHDOWN TIME @ SAME RUNWAY (PREVIOUS LANDING) (MIN.)",column_4,"SPILLAGE\n(0 - NO/\n1 - YES)?","LONGITUDE - HOLDING POINT","LATITUDE - HOLDING POINT","DISTANCE FROM HOLDING POINT TO DXB (NM)","DAY OF WEEK"]).reset_index(drop=True).to_feather(file_name_f)

In [10]:
df = pd.read_feather(file_name_f)

In [11]:
df.head()

,LANDING RUNWAY,ENTRY POINT @ STAR,LONGITUDE - ENTRY 50 NM RADIUS,LATITUDE - ENTRY 50 NM RADIUS,QUADRANT,WAKE TURBULENCE CATEGORY LANDING SEQUENCE @ SAME RUNWAY (PREVIOUS-CURRENT),NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE,NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE,NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (1ST QUADRANT,NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (2ND QUADRANT,...,WAKE TURBULENCE CATEGORY,NUMBER OF AIRCRAFT TAKING-OFF DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE,RWY_12,RWY_30,FLAG HOLDING,HOLDING TIME (MIN.),HOLDING DISTANCE (NM),ALTITUDE - ENTRY 50 NM RADIUS (M),TRUE SPEED - ENTRY 50 NM RADIUS (M/S),METAR VISIBILITY (KM)
0,12L,PUVAL,55.92626538,25.91542267,1,M-M,0.0,0.0,0,0,...,M,0.0,1.0,0.0,1.0,5.400000,23.471750,3988.486488,139.280428,10.0
1,12L,IMPED,56.20236538,24.91151323,4,M-H,1.0,0.0,1,0,...,H,0.0,1.0,0.0,0.0,0.000000,0.000000,3916.68,134.819667,10.0
2,12L,LORID,54.57546538,24.82598604,3,None,2.0,0.0,1,0,...,M,1.0,1.0,0.0,0.0,0.000000,0.000000,3965.57304,165.551079,10.0
3,12L,IMPED,56.21316538,24.93424426,4,J-M,3.0,0.0,1,0,...,M,2.0,1.0,0.0,1.0,6.550000,27.249799,4373.477341,135.064820,10.0
4,12L,PUVAL,55.92566538,25.91584137,1,H-J,4.0,0.0,1,0,...,J,3.0,1.0,0.0,1.0,3.983333,17.591612,3720.86593,140.448066,10.0


In [12]:
df=df.replace("12L",0)
df=df.replace("12R",0)
df=df.replace("30L",1)
df=df.replace("30R",1)
df=df.replace("PUVAL",0)
df=df.replace("VUTEB",1)
df=df.replace("DATOB",1)
df=df.replace("LORID",2)
df=df.replace("IMPED",3)
df=df.replace("L",0)
df=df.replace("M",1)
df=df.replace("H",2)
df=df.replace("J",3)
df

,LANDING RUNWAY,ENTRY POINT @ STAR,LONGITUDE - ENTRY 50 NM RADIUS,LATITUDE - ENTRY 50 NM RADIUS,QUADRANT,WAKE TURBULENCE CATEGORY LANDING SEQUENCE @ SAME RUNWAY (PREVIOUS-CURRENT),NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE,NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE,NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (1ST QUADRANT,NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (2ND QUADRANT,...,WAKE TURBULENCE CATEGORY,NUMBER OF AIRCRAFT TAKING-OFF DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE,RWY_12,RWY_30,FLAG HOLDING,HOLDING TIME (MIN.),HOLDING DISTANCE (NM),ALTITUDE - ENTRY 50 NM RADIUS (M),TRUE SPEED - ENTRY 50 NM RADIUS (M/S),METAR VISIBILITY (KM)
0,0.0,0.0,55.92626538,25.91542267,1,M-M,0.0,0.0,0,0,...,1.0,0.0,1.0,0.0,1.0,5.400000,23.471750,3988.486488,139.280428,10.0
1,0.0,3.0,56.20236538,24.91151323,4,M-H,1.0,0.0,1,0,...,2.0,0.0,1.0,0.0,0.0,0.000000,0.000000,3916.68,134.819667,10.0
2,0.0,2.0,54.57546538,24.82598604,3,None,2.0,0.0,1,0,...,1.0,1.0,1.0,0.0,0.0,0.000000,0.000000,3965.57304,165.551079,10.0
3,0.0,3.0,56.21316538,24.93424426,4,J-M,3.0,0.0,1,0,...,1.0,2.0,1.0,0.0,1.0,6.550000,27.249799,4373.477341,135.064820,10.0
4,0.0,0.0,55.92566538,25.91584137,1,H-J,4.0,0.0,1,0,...,3.0,3.0,1.0,0.0,1.0,3.983333,17.591612,3720.86593,140.448066,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949991,NaN,NaN,None,None,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
949992,NaN,NaN,None,None,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
949993,NaN,NaN,None,None,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
949994,NaN,NaN,None,None,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [13]:
feature_1="LANDING RUNWAY"
#feature_1a="RWY_12"
#feature_1b="RWY_30"
feature_2="LONGITUDE - ENTRY "+str(circle_radius)+" NM RADIUS"
feature_3="LATITUDE - ENTRY "+str(circle_radius)+" NM RADIUS"
feature_4="NUMBER OF AIRCRAFT ENTERING THE "+str(circle_radius)+" NM RADIUS DURING THE "+str(delta_time)+" MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE"
feature_5="NUMBER OF AIRCRAFT TAKING-OFF DURING THE "+str(delta_time)+" MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE"
feature_6="TOTAL NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE"
feature_6a="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (1ST QUADRANT)"
feature_6b="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (2ND QUADRANT)"
feature_6c="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (3RD QUADRANT)"
feature_6d="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (4TH QUADRANT)"
#feature_6a="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (SUPER WAKE TURBULENCE CATEGORY)"
#feature_6b="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (HEAVY WAKE TURBULENCE CATEGORY)"
#feature_6c="NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE (MEDIUM WAKE TURBULENCE CATEGORY)"
feature_7="NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE "+str(delta_time)+" MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE"
feature_8="WAKE TURBULENCE CATEGORY"
feature_9="ALTITUDE - ENTRY "+str(circle_radius)+" NM RADIUS (M)"
feature_10="TRUE SPEED - ENTRY "+str(circle_radius)+" NM RADIUS (M/S)"
feature_11="METAR VISIBILITY (KM)"
feature_12="DELTA TIME ENTRY @ 50 NM CIRCLE FROM PREVIOUS FLIGHT IN THE SAME QUADRANT (MIN.)"
features=[feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11]
#features=[feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_11]
#features=[feature_1,feature_2,feature_3,feature_6,feature_8]
targets=["FLAG HOLDING"]
all_items=features+targets

In [14]:
df = df[df["QUADRANT"].str.contains("-") == False]

In [15]:
df.columns

Index(['LANDING RUNWAY', 'ENTRY POINT @ STAR',
       'LONGITUDE - ENTRY 50 NM RADIUS', 'LATITUDE - ENTRY 50 NM RADIUS',
       'QUADRANT',
       'WAKE TURBULENCE CATEGORY LANDING SEQUENCE @ SAME RUNWAY (PREVIOUS-CURRENT)',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE',
       'NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (1ST QUADRANT',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (2ND QUADRANT',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (3RD QUADRANT',
       'NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (4TH

In [16]:
df

,LANDING RUNWAY,ENTRY POINT @ STAR,LONGITUDE - ENTRY 50 NM RADIUS,LATITUDE - ENTRY 50 NM RADIUS,QUADRANT,WAKE TURBULENCE CATEGORY LANDING SEQUENCE @ SAME RUNWAY (PREVIOUS-CURRENT),NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE,NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE,NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (1ST QUADRANT,NUMBER OF AIRCRAFT ENTERING THE 50 NM RADIUS DURING THE 10.0 MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE (2ND QUADRANT,...,WAKE TURBULENCE CATEGORY,NUMBER OF AIRCRAFT TAKING-OFF DURING THE 10.0 MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE,RWY_12,RWY_30,FLAG HOLDING,HOLDING TIME (MIN.),HOLDING DISTANCE (NM),ALTITUDE - ENTRY 50 NM RADIUS (M),TRUE SPEED - ENTRY 50 NM RADIUS (M/S),METAR VISIBILITY (KM)
0,0.0,0.0,55.92626538,25.91542267,1,M-M,0.0,0.0,0,0,...,1.0,0.0,1.0,0.0,1.0,5.400000,23.471750,3988.486488,139.280428,10.0
1,0.0,3.0,56.20236538,24.91151323,4,M-H,1.0,0.0,1,0,...,2.0,0.0,1.0,0.0,0.0,0.000000,0.000000,3916.68,134.819667,10.0
2,0.0,2.0,54.57546538,24.82598604,3,None,2.0,0.0,1,0,...,1.0,1.0,1.0,0.0,0.0,0.000000,0.000000,3965.57304,165.551079,10.0
3,0.0,3.0,56.21316538,24.93424426,4,J-M,3.0,0.0,1,0,...,1.0,2.0,1.0,0.0,1.0,6.550000,27.249799,4373.477341,135.064820,10.0
4,0.0,0.0,55.92566538,25.91584137,1,H-J,4.0,0.0,1,0,...,3.0,3.0,1.0,0.0,1.0,3.983333,17.591612,3720.86593,140.448066,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189750,0.0,1.0,54.57996538,25.69052004,2,H-M,3.0,0.0,0,1,...,1.0,6.0,1.0,0.0,0.0,0.000000,0.000000,3025.14,199.855956,10.0
189751,0.0,3.0,56.17816538,24.86624067,4,M-H,4.0,1.0,0,2,...,2.0,6.0,1.0,0.0,0.0,0.000000,0.000000,3543.3,177.591255,10.0
189752,0.0,3.0,56.20486538,24.91661458,4,H-M,5.0,1.0,0,2,...,1.0,5.0,1.0,0.0,0.0,0.000000,0.000000,4029.810152,183.246079,10.0
189753,0.0,3.0,56.21196538,24.93162539,4,M-H,5.0,1.0,0,2,...,2.0,5.0,1.0,0.0,0.0,0.000000,0.000000,4014.282107,175.981653,10.0


In [17]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [18]:
df_train_x = df_train.filter(items=features)
df_train_y = df_train.filter(items=targets)

In [19]:
df_test_x = df_test.filter(items=features)
df_test_y = df_test.filter(items=targets)

In [20]:
scaler=MinMaxScaler()
scaler.fit(df_train_x)

MinMaxScaler()

In [21]:
df_train_x=scaler.transform(df_train_x)
df_test_x=scaler.transform(df_test_x)

In [22]:
df_train_y=df_train_y.values.flatten()

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [23]:
model=DecisionTreeClassifier(max_depth=12)
#model=KNeighborsClassifier(n_neighbors=10,weights='distance',leaf_size=60)
#model=RandomForestClassifier(n_estimators=300,max_depth=20)
#model=MLPClassifier(max_iter=300,activation='relu',solver='adam',random_state=1)
#model=GradientBoostingClassifier()
#model=ctb.CatBoostClassifier(iterations=1000,learning_rate=1,depth=2)

In [24]:
model.fit(df_train_x, df_train_y)

DecisionTreeClassifier(max_depth=12)

In [25]:
y_pred = model.predict(df_test_x)

In [26]:
y_pred

array([0., 1., 1., ..., 0., 0., 1.])

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn-metrics-confusion-matrix

In [27]:
confusion_matrix(df_test_y, y_pred)

array([[28941,   905],
       [ 1743,  6362]], dtype=int64)

In [28]:
precision_score(df_test_y, y_pred, average=None)

array([0.94319515, 0.87546443])

In [29]:
precision_score(df_test_y, y_pred)

0.8754644282372368

In [30]:
recall_score(df_test_y, y_pred, average=None)

array([0.96967768, 0.78494756])

In [31]:
recall_score(df_test_y, y_pred)

0.7849475632325725

In [32]:
accuracy_score(df_test_y, y_pred)

0.9302258175015151

In [33]:
f1_score(df_test_y, y_pred, average=None)

array([0.9562531 , 0.82773875])

In [34]:
f1_score(df_test_y, y_pred)

0.8277387457715327

In [35]:
# To save the model
file_name_pkl="Final Summary - STAR - DXB - ({rd:.0f} NM) Revision 03 ({dt:.1f} minutes).pkl"
file_name_pkl=file_name_pkl.format(dt=delta_time,rd=circle_radius)
model_name=file_name_pkl
with open(model_name,'wb') as f:
    pickle.dump(model,f)

In [36]:
# To load the model and run an analysis
with open(model_name, 'rb') as f:
    model=pickle.load(f)

In [37]:
df_x=df.filter(items=features)
df_x=scaler.transform(df_x)
df_x_results=model.predict(df_x)
df_x_results_array=np.array(df_x_results)

In [38]:
file_name_csv="Final Summary - STAR - DXB - ({rd:.0f} NM) Revision 03 ({dt:.1f} minutes).csv"
file_name_csv=file_name_pkl.format(dt=delta_time,rd=circle_radius)
results_file=file_name_csv
np.savetxt(results_file,df_x_results_array,delimiter=",")

In [39]:
differences_array=abs(df_x_results-df["FLAG HOLDING"])
total_differences=sum(differences_array)
num_rows = np.shape(differences_array)[0]

In [40]:
percentage_errors=total_differences/num_rows
100*percentage_errors

5.516060182867381